In [1]:
import os
from dotenv import load_dotenv

#환경변수 로드
load_dotenv()

True

In [2]:
from langchain_neo4j import Neo4jGraph

#LangChian도구 활용 - DB 연결 객체 초기화
graph = Neo4jGraph(
    url = os.getenv("NEO4J_URI"),
    username = os.getenv("NEO4J_USERNAME"),
    password = os.getenv("NEO4J_PASSWORD"),
    database = os.getenv("NEO4J_DATABASE"),
    enhanced_schema = True,
)

In [3]:
#테스트 쿼리 실행
cypher_query = """
CREATE (n:Test {name: "Hello Neo4j Desktop News DB"})
RETURN n
"""

graph.query(cypher_query)

[{'n': {'name': 'Hello Neo4j Desktop News DB'}}]

### 2.1 뉴스 데이터 전처리

#### 1) 데이터셋 준비

In [4]:
#뉴스 데이터 로드
def load_news_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        news_article = file.read()
    return news_article

#뉴스 데이터 목록 확인(glob)
import glob
news_files = sorted(glob.glob(os.path.join('news_data','*.md')))

#뉴스 데이터 목록 출력
print("뉴스 데이터 목록:")
for news_file in news_files:
    print(os.path.basename(news_file))

#뉴스 데이터 로드 및 정리
news_articles =[]
for news_file in news_files:
    news_article = load_news_data(news_file)
    news_articles.append(news_article)

#뉴스 데이터 출력
print("\n 뉴스 데이터:")
for i , news_article in enumerate(news_articles):
    print(f"\n뉴스 기사 {i+1}:\n{news_article[:100]}...") # 첫 100자만 출력
    print("-"*100)

뉴스 데이터 목록:
news_article_1.md
news_article_2.md
news_article_3.md
news_article_4.md
news_article_5.md

 뉴스 데이터:

뉴스 기사 1:
# 구글, 안드로이드 14에 생성형 AI 기능 대거 탑재

**디지털타임스 | 박승리 기자 | 2024-03-20**

구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고...
----------------------------------------------------------------------------------------------------

뉴스 기사 2:
# 애플, iOS 18 공개하며 AI 플랫폼 경쟁 본격화

**디지털타임스 | 최기상 기자 | 2024-03-19**

애플이 차세대 모바일 운영체제 iOS 18을 공개하며 AI ...
----------------------------------------------------------------------------------------------------

뉴스 기사 3:
# 메타, 차세대 오픈소스 AI 모델 'Llama 3' 공개

**디지털타임스 | 정주리 기자 | 2024-04-05**

메타가 차세대 오픈소스 AI 모델 'Llama 3'를 공...
----------------------------------------------------------------------------------------------------

뉴스 기사 4:
# 삼성전자, 'AI 메모리' 신기술 개발 성공

**전자일보 | 이승지 기자 | 2024-04-03**

삼성전자가 AI 연산에 최적화된 '컴퓨팅 인 메모리'(CIM) 기술을 적...
----------------------------------------------------------------------------------------------------

뉴스 기사 5:
# 엔비디아, 차세대 AI 전용 G

#### 2) 뉴스 메타데이터 추출

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field
from datetime import date

#뉴스 기사 메타데이터 모델 정의
class NewsArticleMetadata(BaseModel):
    """뉴스 기사의 구조화된 메타 데이터"""
    title: str = Field(description="뉴스 기사의 제목")
    source : str = Field(description = "뉴스 출처/발행처")
    author : str = Field(description = "기사 작성자의 이름 (직위 등 표시 불필요)")
    date : str = Field(description = "발행일(YYYY-MM-DD 형식)")
    content : str = Field (description = "기사의 전체 텍스트 내용")

def extract_article_metadata(article_text : str) -> NewsArticleMetadata:
    """
    Langchain을 사용하여 뉴스 기사에서 구조화된 메타데이터를 추출합니다.
    
    Args:
        article_text (str): 뉴스 기사의 전체 텍스트

    Returns :
        NewsArticleMetadata : 기사에서 추출한 구조화된 메타데이터
    """

    #프롬프트 템플릿 정의 - LLM에게 메타데이터 추출 지시사항 제공
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert news article metadata extractor. 
        Extract precise and accurate information from the given news article.
        
        Extraction Guidelines:
        - Identify the most accurate title
        - Determine the primary source/publication
        - Extract the author's name (only name, no position)
        - Identify the publication date
        - List key organizations mentioned
        - Highlight key technologies discussed
        
        Be as specific and factual as possible."""),
        ("human", """Extract metadata from the following article:\n\n{article_text}""")
    ])


    llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature = 0)

    #구조화된 출력을 위해 Pydantic 모델과 연결
    llm_with_structured_output = llm.with_structured_output(NewsArticleMetadata)

    chain = prompt | llm_with_structured_output

    #메타데이터 추출 시도
    try:
        #LLM을 통해 메타데이터 추출
        metadata = chain.invoke({
            "article_text": article_text,
        })
        #데이터 누락시, 기본값 설정 - 데이터 일관성 유지
        if not metadata.title :
            metadata.title = ""
        if not metadata.source :
            metadata.source = "미상"
        if not metadata.author :
            metadata.author = "미상"
        if not metadata.date :
            metadata.date = "미상"
        if not metadata.content :
            metadata.content = article_text
        
        return metadata
    
    except Exception as e:
        #에러 발생시 로그 출력 및 기본 메타데이터 반환
        print(f"메타데이터 추출 중 오류 발생: {e}")
        return NewsArticleMetadata(
            title="",
            source = "미상",
            author = "미상",
            date = "미상",
            content = ""
        )

#모든 뉴스 기사에서 메타데이터 추출 및 결과 출력
extracted_metadata = []
for article in news_articles:
    #각 기사에서 메타데이터 추출
    metadata = extract_article_metadata(article)
    #추출된 메타데이터 저장
    extracted_metadata.append(metadata)
    #추출 결과 출력 - 디버깅 및 확인용
    print(f"Extracted metadata for article: {metadata.title}")
    print(f"Source: {metadata.source}")
    print(f"Author: {metadata.author}")
    print(f"Date: {metadata.date}")
    print(f"Content: {metadata.content[:100]}...") #첫 100자만 출력하여 가독성 확보


Extracted metadata for article: 구글, 안드로이드 14에 생성형 AI 기능 대거 탑재
Source: 디지털타임스
Author: 박승리
Date: 2024-03-20
Content: 구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의 인공지능 기술력을 한층 강화하는 전략의 일환으로 보인다. 특히 카메라 기능에서 AI를 활...
Extracted metadata for article: 애플, iOS 18 공개하며 AI 플랫폼 경쟁 본격화
Source: 디지털타임스
Author: 최기상
Date: 2024-03-19
Content: 애플이 차세대 모바일 운영체제 iOS 18을 공개하며 AI 플랫폼 경쟁을 본격화하고 있다. 최근 샌프란시스코에서 열린 WWDC 2024에서 공개된 iOS 18은 애플의 인공지능 전...
Extracted metadata for article: 메타, 차세대 오픈소스 AI 모델 'Llama 3' 공개
Source: 디지털타임스
Author: 정주리
Date: 2024-04-05
Content: 메타가 차세대 오픈소스 AI 모델 'Llama 3'를 공개했다. 이번 모델은 이전 버전보다 2배 큰 파라미터 규모와 10배 많은 학습 데이터를 기반으로 개발되었다고 회사 측은 밝혔...
Extracted metadata for article: 삼성전자, 'AI 메모리' 신기술 개발 성공
Source: 전자일보
Author: 이승지
Date: 2024-04-03
Content: 삼성전자가 AI 연산에 최적화된 '컴퓨팅 인 메모리'(CIM) 기술을 적용한 신개념 메모리 반도체 개발에 성공했다고 발표했다. 이번 기술은 기존 메모리와 CPU 간의 데이터 이동 ...
Extracted metadata for article: 엔비디아, 차세대 AI 전용 GPU 'H200' 출시 임박
Source: 전자일보
Author: 김송이
Date: 2024-04-02
Content: 엔비디아가 차세대 AI 

In [25]:
extracted_metadata[0].model_dump()

{'title': '구글, 안드로이드 14에 생성형 AI 기능 대거 탑재',
 'source': '디지털타임스',
 'author': '박승리',
 'date': '2024-03-20',
 'content': '구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의 인공지능 기술력을 한층 강화하는 전략의 일환으로 보인다. 특히 카메라 기능에서 AI를 활용한 사진 보정 기능이 크게 개선되었다. 사용자가 촬영한 사진을 인공지능이 자동으로 분석하여 최적의 색감과 구도로 보정해주는 기능이 추가되었으며, 이는 전문 사진작가 수준의 결과물을 일반 사용자도 쉽게 얻을 수 있게 해준다. 또한 실시간 번역 기능도 대폭 강화되었다. 이제 사용자들은 외국어 텍스트를 카메라로 비추기만 해도 즉시 번역 결과를 확인할 수 있으며, 통화 중에도 실시간으로 음성을 번역해주는 기능이 추가되었다. 구글 어시스턴트 역시 생성형 AI 기술을 적용해 더 자연스러운 대화가 가능해졌다. 이전보다 더 복잡한 질문에도 정확한 답변을 제공하며, 사용자의 습관과 선호도를 학습하여 개인화된 서비스를 제공한다. 구글 안드로이드 부문 책임자는 "이번 안드로이드 14에 탑재된 생성형 AI 기능은 사용자 경험을 혁신적으로 개선할 것"이라며 "앞으로도 인공지능 기술을 통해 더 편리하고 스마트한 모바일 경험을 제공하기 위해 노력할 것"이라고 밝혔다. 업계 전문가들은 이번 구글의 AI 기능 강화가 모바일 OS 시장에서의 경쟁력을 높이는 중요한 전환점이 될 것으로 전망하고 있다.'}

In [26]:
#메타데이터를 JSONL 형식으로 저장
import json
with open('news_data/news_metadata.jsonl','w', encoding='utf-8') as f:
    for metadata in extracted_metadata:
        json.dump(metadata.model_dump(),f,ensure_ascii=False)
        f.write('\n')

In [27]:
#JSONL 파일 확인
import json
with open('news_data/news_metadata.jsonl','r', encoding='utf-8') as f:
    news_metadata = [json.loads(line) for line in f]
    print("뉴스 메타데이터:")
    for i, metadata in enumerate(news_metadata):
        print(f"\n뉴스 기사 {i+1}")
        print(f"Title {metadata['title']}")
        print(f"Source {metadata['source']}")
        print(f"Author {metadata['author']}")
        print(f"Date {metadata['date']}")
        print(f"Content {metadata['content'][:100]}...")
        print("-"*100)

뉴스 메타데이터:

뉴스 기사 1
Title 구글, 안드로이드 14에 생성형 AI 기능 대거 탑재
Source 디지털타임스
Author 박승리
Date 2024-03-20
Content 구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의 인공지능 기술력을 한층 강화하는 전략의 일환으로 보인다. 특히 카메라 기능에서 AI를 활...
----------------------------------------------------------------------------------------------------

뉴스 기사 2
Title 애플, iOS 18 공개하며 AI 플랫폼 경쟁 본격화
Source 디지털타임스
Author 최기상
Date 2024-03-19
Content 애플이 차세대 모바일 운영체제 iOS 18을 공개하며 AI 플랫폼 경쟁을 본격화하고 있다. 최근 샌프란시스코에서 열린 WWDC 2024에서 공개된 iOS 18은 애플의 인공지능 전...
----------------------------------------------------------------------------------------------------

뉴스 기사 3
Title 메타, 차세대 오픈소스 AI 모델 'Llama 3' 공개
Source 디지털타임스
Author 정주리
Date 2024-04-05
Content 메타가 차세대 오픈소스 AI 모델 'Llama 3'를 공개했다. 이번 모델은 이전 버전보다 2배 큰 파라미터 규모와 10배 많은 학습 데이터를 기반으로 개발되었다고 회사 측은 밝혔...
----------------------------------------------------------------------------------------------------

뉴스 기사 4
Title 삼성전자, 'AI 메모리' 신기술 개발 성공
Source 전자일보
Author 이승지
Date 2024-04-03
Content 삼성전자

#### 3) 뉴스 데이터 구조화

In [28]:
# 뉴스 데이터 구조화 - JSONL 파일에서 로드한 메타데이터를 분석 가능한 형태로 변환

extracted_articles = []  # 구조화된 뉴스 기사를 저장할 빈 리스트 초기화

# 각 뉴스 메타데이터를 순회하면서 구조화된 형태로 변환
for i, article in enumerate(news_metadata):
    
    # 개별 기사 데이터 구조화 - 일관된 형식으로 데이터 정리
    # - id: 고유 식별자 부여 (article_0, article_1 등의 형식)
    # - 제목, 출처, 작성자, 날짜, 내용 등 핵심 정보 포함
    article = {
        "id": f"article_{i}",  # 고유 식별자 생성 (인덱스 기반)
        "title": article["title"],  # 기사 제목
        "source": article["source"],  # 기사 출처 (언론사)
        "author": article["author"],  # 기사 작성자
        "date": article["date"],  # 기사 발행일
        "content": article["content"]  # 기사 전체 내용
    }

    # 구조화된 기사 데이터를 리스트에 추가 - 이후 지식 그래프 구축에 활용
    extracted_articles.append(article)

# 구조화된 기사 데이터 확인 - 처리 결과 검증 및 디버깅
print("구조화된 뉴스 기사 데이터:")
for article in extracted_articles:
    print(f"ID: {article['id']}")  # 고유 식별자 출력
    print(f"Title: {article['title']}")  # 기사 제목 출력
    print(f"Source: {article['source']}")  # 출처 출력
    print(f"Author: {article['author']}")  # 작성자 출력
    print(f"Date: {article['date']}")  # 날짜 출력
    print(f"Content: {article['content'][:100]}...")  # 내용 일부만 출력하여 가독성 확보
    print("-" * 100)

구조화된 뉴스 기사 데이터:
ID: article_0
Title: 구글, 안드로이드 14에 생성형 AI 기능 대거 탑재
Source: 디지털타임스
Author: 박승리
Date: 2024-03-20
Content: 구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의 인공지능 기술력을 한층 강화하는 전략의 일환으로 보인다. 특히 카메라 기능에서 AI를 활...
----------------------------------------------------------------------------------------------------
ID: article_1
Title: 애플, iOS 18 공개하며 AI 플랫폼 경쟁 본격화
Source: 디지털타임스
Author: 최기상
Date: 2024-03-19
Content: 애플이 차세대 모바일 운영체제 iOS 18을 공개하며 AI 플랫폼 경쟁을 본격화하고 있다. 최근 샌프란시스코에서 열린 WWDC 2024에서 공개된 iOS 18은 애플의 인공지능 전...
----------------------------------------------------------------------------------------------------
ID: article_2
Title: 메타, 차세대 오픈소스 AI 모델 'Llama 3' 공개
Source: 디지털타임스
Author: 정주리
Date: 2024-04-05
Content: 메타가 차세대 오픈소스 AI 모델 'Llama 3'를 공개했다. 이번 모델은 이전 버전보다 2배 큰 파라미터 규모와 10배 많은 학습 데이터를 기반으로 개발되었다고 회사 측은 밝혔...
----------------------------------------------------------------------------------------------------
ID: article_3
Title: 삼성전자, 'AI 메모리' 신기술 개발 성공
Source: 전

### 2.2 KG 온톨로지 구현

In [29]:
constraints=[
    "CREATE CONSTRAINT IF NOT EXISTS FOR (n:NewsArticle) REQUIRE n.id IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Company) REQUIRE c.name IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Product) REQUIRE p.name IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (t:Technology) REQUIRE t.name IS UNIQUE",
    "CREATE CONSTRAINT IF NOT EXISTS FOR (p:Person) REQUIRE p.name IS UNIQUE",
]

for constraint in constraints:
    graph.query(constraint)

In [ ]:
graph.query("SHOW CONSTRAINTS") #현재 설정된 제약조건 확인

[{'id': 3,
  'name': 'constraint_5b467fef',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['NewsArticle'],
  'properties': ['id'],
  'ownedIndex': 'constraint_5b467fef',
  'propertyType': None},
 {'id': 9,
  'name': 'constraint_7ef39459',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['Technology'],
  'properties': ['name'],
  'ownedIndex': 'constraint_7ef39459',
  'propertyType': None},
 {'id': 11,
  'name': 'constraint_a831e4ce',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['Person'],
  'properties': ['name'],
  'ownedIndex': 'constraint_a831e4ce',
  'propertyType': None},
 {'id': 5,
  'name': 'constraint_bd66231e',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['Company'],
  'properties': ['name'],
  'ownedIndex': 'constraint_bd66231e',
  'propertyType': None},
 {'id': 7,
  'name': 'constraint_f7313309',
  'type': 'UNIQUENESS',
  'entityType': 'NODE',
  'labelsOrTypes': ['Product'],
  'properties':

#### 3) 뉴스 기사에서 엔티티와 관계를 추출

In [30]:
from typing import List, Dict, Any
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document

#LLM설정
llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0)

#뉴스 기사에서 추출할 엔티티 타입을 제한하여 정확도를 향상시킨다.
allowed_nodes = ["Company","Product","Technology"]

allowed_relationships = [
    ("Company", "RELEASED", "Product"), #회사가 제품을 출시함
    ("Company","DEVELOPED","Technology"), #회사가 기술을 개발함
    ("Product","USES","Technology") #제품이 기술을 사용함
]
#텍스트에서 그래프 구조(노드와 관계)를 추출하는 변환기 설정
transformer = LLMGraphTransformer(
    llm = llm,
    allowed_nodes = allowed_nodes, #허용되는 노드 유형
    allowed_relationships = allowed_relationships, #허용되는 관계 유형
    node_properties =["industry","version","releaseDate","category"] #노드에 추가할 수 있는 속성들
)

In [31]:
#뉴스 기사 데이터를 LangChain의 Document 객체로 변환하는 함수
# - LangChain의 Document 객체는 텍스트 콘텐츠와 메타데이터를 함께 저장할 수 있는 구조
# - 이를 통해 LLM이 문서 처리 시 메타데이터 정보도 함께 활용 가능

def convert_to_documents(articles: List[Dict[str,Any]]) -> List[Document]:
    documents=[]
    for article in articles:
        #각 기사마다 Document 객체 생성
        doc = Document(
            page_content = article["content"],
            metadata={
                "id": article["id"],
                "title": article["title"],
                "source": article["source"],
                "author": article["author"],
                "date": article["date"]
            }
        )
        documents.append(doc)
    return documents

documents = convert_to_documents(extracted_articles)
print(f"변환된 문서 수: {len(documents)}")
print("="*100)

print(documents[0].page_content[:100])
print("-"*100)
print(documents[0].metadata)

변환된 문서 수: 5
구글이 안드로이드 14에 생성형 AI 기능을 대거 탑재했다고 발표했다. 이번 발표는 구글의 인공지능 기술력을 한층 강화하는 전략의 일환으로 보인다. 특히 카메라 기능에서 AI를 활
----------------------------------------------------------------------------------------------------
{'id': 'article_0', 'title': '구글, 안드로이드 14에 생성형 AI 기능 대거 탑재', 'source': '디지털타임스', 'author': '박승리', 'date': '2024-03-20'}


In [33]:
# 뉴스 기사에서 그래프 데이터 추출
# - LLMGraphTransformer를 사용하여 일반 문서를 그래프 구조로 변환
# - 각 문서(뉴스 기사)에서 노드(회사, 기술, 제품 등)와 관계(개발, 출시, 사용 등)를 추출
# - LLM이 텍스트를 분석하여 허용된 노드 유형과 관계 유형에 맞게 구조화된 데이터 생성
# - 추출된 그래프 데이터는 Neo4j에 저장하기 위한 중간 형태로 사용됨
graph_documents = transformer.convert_to_graph_documents(documents)
print(f"그래프 문서 수: {len(graph_documents)}")

그래프 문서 수: 5


In [ ]:
graph_documents[0].nodes #첫번째 그래프 문서의 노드 정보 확인

[Node(id='Google', type='Company', properties={}),
 Node(id='Android 14', type='Product', properties={'version': '14'}),
 Node(id='Generative Ai', type='Technology', properties={}),
 Node(id='Ai-Powered Photo Correction', type='Technology', properties={'category': 'camera feature'}),
 Node(id='Real-Time Translation', type='Technology', properties={'category': 'language feature'}),
 Node(id='Google Assistant', type='Product', properties={})]

In [ ]:
graph_documents[0].relationships # 첫번째 그래프 문서의 관계 정보 확인

[Relationship(source=Node(id='Google', type='Company', properties={}), target=Node(id='Android 14', type='Product', properties={}), type='RELEASED', properties={}),
 Relationship(source=Node(id='Android 14', type='Product', properties={}), target=Node(id='Generative Ai', type='Technology', properties={}), type='USES', properties={}),
 Relationship(source=Node(id='Android 14', type='Product', properties={}), target=Node(id='Ai-Powered Photo Correction', type='Technology', properties={}), type='USES', properties={}),
 Relationship(source=Node(id='Android 14', type='Product', properties={}), target=Node(id='Real-Time Translation', type='Technology', properties={}), type='USES', properties={}),
 Relationship(source=Node(id='Google Assistant', type='Product', properties={}), target=Node(id='Generative Ai', type='Technology', properties={}), type='USES', properties={}),
 Relationship(source=Node(id='Google', type='Company', properties={}), target=Node(id='Generative Ai', type='Technology', p

In [36]:
# 추출된 그래프 데이터를 Neo4j 데이터베이스에 저장하는 과정

for doc in graph_documents:
    print(f"추출된 노드: {doc.nodes}")
    print(f"추출된 관계: {doc.relationships}")
    graph.add_graph_documents([doc],include_source=True)
    
    # Neo4j 그래프 데이터베이스에 추출된 정보 저장
    # - add_graph_documents: 그래프 문서를 Neo4j에 저장하는 메서드
    # - [doc]: 단일 문서를 리스트 형태로 전달 (일괄 처리 가능)
    # - include_source=True: 원본 문서 정보도 함께 저장 (추적성 유지)
    print("-"*100)


추출된 노드: [Node(id='Google', type='Company', properties={}), Node(id='Android 14', type='Product', properties={'version': '14'}), Node(id='Generative Ai', type='Technology', properties={}), Node(id='Ai-Powered Photo Correction', type='Technology', properties={'category': 'camera feature'}), Node(id='Real-Time Translation', type='Technology', properties={'category': 'language feature'}), Node(id='Google Assistant', type='Product', properties={})]
추출된 관계: [Relationship(source=Node(id='Google', type='Company', properties={}), target=Node(id='Android 14', type='Product', properties={}), type='RELEASED', properties={}), Relationship(source=Node(id='Android 14', type='Product', properties={}), target=Node(id='Generative Ai', type='Technology', properties={}), type='USES', properties={}), Relationship(source=Node(id='Android 14', type='Product', properties={}), target=Node(id='Ai-Powered Photo Correction', type='Technology', properties={}), type='USES', properties={}), Relationship(source=Node(

In [39]:
cypher_query="""
MATCH (d:Document)
SET d:NewsArticle
REMOVE d:Document
RETURN count(d) AS count
"""

result = graph.query(cypher_query)

print(f"모든 Document 노드 이름을 NewsArticle로 변경했습니다. 변경되 노드 수: {result[0]['count']}")

모든 Document 노드 이름을 NewsArticle로 변경했습니다. 변경되 노드 수: 0


#### 4) NewsArticle 관련 노드/관계 생성

In [40]:
# 언론사를 Company 노드로 추가하는 Cypher 쿼리

query ="""
MATCH (n:NewsArticle)
WITH DISTINCT n.source AS source
MERGE (c:Compnay {name: source, type : 'news'})
RETURN count(c) AS count
"""

#Neo4J 데이터베이스에 쿼리 실행
result = graph.query(query)
print(f"언론사 노드수: {result[0]['count']}")

언론사 노드수: 2


In [42]:
# 뉴스 기사와 언론사 노드 연결하는 Cypher 쿼리
query = """
MATCH (n:NewsArticle), (c:Company)
WHERE n.source = c.name
MERGE (n)-[:PUBLISHED_BY]->(c)
RETURN count(n) AS count
"""
# Neo4j 데이터베이스에 쿼리 실행
result = graph.query(query)
# 생성된 관계 수 출력
print(f"뉴스 기사와 언론사 노드 연결 수: {result[0]['count']}")

뉴스 기사와 언론사 노드 연결 수: 0


In [43]:
#기자를 Person노드로 추가하고, 뉴스 기사 노드와 언론사 노드와 연결하는 Cypher 쿼리
query = """
MATCH (n:NewsArticle)
WITH DISTINCT n.author AS author, n.source AS source, n.id AS articleId

//MERGE : 노드가 존재하지 않으면 생성하고, 존재하면 매칭함
MERGE (p:Person {name: author})
WITH p,source,articleId
MERGE (c:Company {name:source})
MERGE (p)-[:WORKS_FOR]->(c)
WITH p, articleId
MATCH (a: NewsArticle {id: articleId})
MERGE(p)-[:WROTE]->(a)
RETURN count(DISTINCT p) AS count
"""

result = graph.query(query)
print(f"기자 노드 수: {result[0]['count']}")

기자 노드 수: 5
